In [13]:
# %load /home/gaetan/Desktop/geovpylib/templates/heading.py
%load_ext autoreload
%autoreload 2

# Common imports
import os
import pandas as pd, numpy as np
import datetime
#import json
import requests
#import duckdb
#import plotly.express as px

# Geovpylib library
import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.decorators as d
import geovpylib.magics
import geovpylib.pks as pks
import geovpylib.queries as q
import geovpylib.record_linkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u
eta = u.Eta()

# Specific imports
# ...

# Global variables
# ...

# Connect to Geovistory database
# env = 'prod' # Database to query: "prod", "stag", "dev", "local"
# pk_project = pks.projects. # The project to query/insert: integer
# execute = False # Boolean to prevent to execute directly into databases
# metadata_str = '' # kebab-lower-case or snake-lower-case. 
# import_manner = 'one-shot' # 'one-shot' or 'batch'
# db.connect_geovistory(env, pk_project, execute)
# db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
# db.set_insert_manner(import_manner)

# Connect to other database
# db_url_env_var_name = 'YELLOW-' # Name of an environment variable holding the Postgres database URL
# execute = False # Boolean to prevent to execute directly into databases
# db.connect_external(os.getenv(db_url_env_var_name), execute=False)

# Connect to a SPARQL endpoint
# sparql.connect_external('url')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# HLS Person

## 1./ Harvest data

### 1.1/ Wikidata

In [12]:
sparql.connect_external('https://query.wikidata.org/sparql')

hls_persons = sparql.query("""
    SELECT ?hls_id ?uri_wikidataLabel ?uri_wikidata ?genderLabel ?description
    WHERE {
        ?uri_wikidata wdt:P31 wd:Q5 .
        ?uri_wikidata wdt:P902 ?hls_id .
        optional { ?uri_wikidata wdt:P21 ?gender . }
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

        SERVICE wikibase:label { 
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . 
            ?uri_wikidata schema:description ?description .
        }
    }
""")
                                
hls_persons.rename(columns={'uri_wikidataLabel': 'name', 'genderLabel': 'gender', 'description': 'definition'}, inplace=True)
hls_persons['uri_hls'] = 'https://hls-dhs-dss.ch/articles/' + hls_persons['hls_id']
hls_persons = hls_persons[['hls_id', 'name', 'gender', 'definition', 'uri_wikidata', 'uri_hls']]
hls_persons['gender'] = hls_persons['gender'].replace('male', 'Male')
hls_persons['gender'] = hls_persons['gender'].replace('female', 'Female')

a.infos(hls_persons, random=True)

# 22s

Shape:  (23967, 6) - extract:


,hls_id,name,gender,definition,uri_wikidata,uri_hls
969,009133,Vincino Beretta,Male,(1920-1972),http://www.wikidata.org/entity/Q51232570,https://hls-dhs-dss.ch/articles/009133
23783,032572,Alfred Wyser,Male,NaN,http://www.wikidata.org/entity/Q96413853,https://hls-dhs-dss.ch/articles/032572
9219,005675,Hans Stähli,Male,Swiss politician (1889-1963),http://www.wikidata.org/entity/Q125346,https://hls-dhs-dss.ch/articles/005675
5219,032407,Giuseppe Gorani,Male,Italian writer (1740-1819),http://www.wikidata.org/entity/Q218342,https://hls-dhs-dss.ch/articles/032407
2694,031206,Isaac Gamaliel de Rovéréa,Male,NaN,http://www.wikidata.org/entity/Q99279014,https://hls-dhs-dss.ch/articles/031206


### 1.2/ metagrid.ch

In [28]:

def get_meta_grid_uris(args):
    """Given the name and some clues (to differentiate between entities that have the same name), fetch the other uris that metagrid has on an entity."""
    name, clues = args

    url = "https://api.metagrid.ch/search"
    response = requests.get(url, params={"group":1, "query":name}).json()

    if 'concordances' not in response: return pd.NA

    for concordance in response['concordances']:
        uris = []
        found = False
        for resource in concordance['resources']:
            uri = resource['link']['uri']
            clues_present = True
            for clue in clues:
                if uri.find(clue) == -1: clues_present = False
            if clues_present: found = True
            uris.append(uri)
        if found: return ' ; '.join(uris)

    return pd.NA

# Test
# get_meta_grid_uris('Camille Guggenheim', ['hls-dhs-dss.ch', '043800'])

In [29]:
from multiprocessing import Pool

with Pool(5) as p:
    iterrable = [(row['name'], ['hls-dhs-dss.ch', row['hls_id']]) for _,row in hls_persons.iterrows()]
    hls_persons['same_as'] = p.map(get_meta_grid_uris, iterrable)
    # uris = p.map(get_meta_grid_uris, iterrable)

In [30]:
hls_persons

,hls_id,name,gender,definition,uri_wikidata,uri_hls,same_as
0,028377,Anatoly Lunacharsky,Male,Russian Marxist revolutionary (1875-1933),http://www.wikidata.org/entity/Q18809,https://hls-dhs-dss.ch/articles/028377,"[https://dodis.ch/P37363, https://hls-dhs-dss...."
1,044295,Louis de Rougemont,Male,explorer with false claims,http://www.wikidata.org/entity/Q20014,https://hls-dhs-dss.ch/articles/044295,[https://www.bsg.nb.admin.ch/discovery/search?...
2,028424,Élisée Reclus,Male,French geographer and writer,http://www.wikidata.org/entity/Q20951,https://hls-dhs-dss.ch/articles/028424,[http://id.loc.gov/authorities/names/n81018103...
3,011946,Hermann Hesse,Male,German writer (1877–1962),http://www.wikidata.org/entity/Q25973,https://hls-dhs-dss.ch/articles/011946,[https://www.bsg.nb.admin.ch/discovery/search?...
4,012028,Paul Klee,Male,Swiss artist (1879-1940),http://www.wikidata.org/entity/Q44007,https://hls-dhs-dss.ch/articles/012028,[https://www.bsg.nb.admin.ch/discovery/search?...
...,...,...,...,...,...,...,...
23962,033193,Emil Neff-Fässler,Male,NaN,http://www.wikidata.org/entity/Q112578600,https://hls-dhs-dss.ch/articles/033193,"[http://viaf.org/viaf/309652424, https://data...."
23963,031874,Walter Thalmann,Male,NaN,http://www.wikidata.org/entity/Q113681577,https://hls-dhs-dss.ch/articles/031874,"[https://d-nb.info/gnd/127985492, https://hls-..."
23964,049335,Rudolf J. Ramseyer,Male,1923-04-03--2007,http://www.wikidata.org/entity/Q113810155,https://hls-dhs-dss.ch/articles/049335,"[http://viaf.org/viaf/75196169, http://www.idr..."
23965,021068,Caspar Diethelm,NaN,NaN,http://www.wikidata.org/entity/Q121761272,https://hls-dhs-dss.ch/articles/021068,[http://www.briefedition.alfred-escher.ch/kont...


### 1.3/ Geovistory

## 2./ Record linkage

## 3./ Import